In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting.probabilistic import (
    PinballLoss,
)
from sktime.split import (
    ExpandingWindowSplitter,
    temporal_train_test_split,
)
from sktime.utils.plotting import plot_windows

from probafcst.models import BenchmarkForecaster
from probafcst.plotting import plot_quantiles

sns.set_theme(style="ticks")

In [ ]:
bikes = pd.read_parquet("../data/bikes.parquet")
load = pd.read_parquet("../data/energy.parquet")

In [ ]:
load.tail()

In [ ]:
# get 3 years as hours


In [ ]:
splitter = ExpandingWindowSplitter(
    fh=ForecastingHorizon(np.arange(1, 24 * 7)),
    initial_window=24 * 365 * 3,  # 3 years
    step_length=24 * 30 * 3,
)
len(list(splitter.split(load)))

In [ ]:
plot_windows(splitter, load)

In [ ]:
y_train, y_test = temporal_train_test_split(load, test_size=24 * 7)
fh = ForecastingHorizon(y_test.index, is_relative=False)
# forecaster = NaiveForecaster(strategy="mean", sp=24, window_length=24 * 7 * 3)
forecaster = BenchmarkForecaster(n_weeks=100)
forecaster.fit(y_train)

quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]
pred_quantiles = forecaster.predict_quantiles(fh, alpha=quantile_levels)
pred_quantiles.head()

In [ ]:
forecaster.predict_interval(fh, coverage=[0.5, 0.95])

In [ ]:
# forecaster = NaiveForecaster(strategy="mean", sp=24 * 7)
results = evaluate(
    forecaster,
    splitter,
    load,
    strategy="refit",
    scoring=PinballLoss(alpha=quantile_levels),
    return_data=True,
)
results.head()

In [ ]:
results.iloc[:, :-3]

In [ ]:
# plot each forecast period
nrows = min(3, len(results))
for i, row in results.iloc[-nrows:].iterrows():
    plot_quantiles(row.y_test, row.y_pred_quantiles)